**Import thư viện**


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

**Đọc và chuẩn  bị dữ liệu**

In [ ]:
# 1. Đọc dữ liệu
df = pd.read_csv('./data/cleaned/hcmhouseprice.csv').dropna()
X = df[['bedrooms', 'bathrooms']].values
y = df['price'].values.reshape(-1, 1)

# Chuẩn hoá dữ liệu để mô hình học ổn định hơn
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

Đặc trưng chọn: Bedrooms - Bathrooms

In [ ]:
# 2. K-Fold setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 3. Tạo lớp Linear Regression bằng PyTorch
class LinearRegressionModel(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.linear(x)

# 4. Cross-validation thủ công
mae_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Tách dữ liệu
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Chuyển sang tensor
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)

    # Khởi tạo model, loss, optimizer
    model = LinearRegressionModel(in_features=2)
    criterion = nn.L1Loss()  # MAE loss
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # Huấn luyện
    epochs = 500
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    # Đánh giá
    model.eval()
    with torch.no_grad():
        preds = model(X_val)
        mae = torch.mean(torch.abs(preds - y_val)).item()
        mae_scores.append(mae)
    last_model = model

# 5. Kết quả trung bình

print("Model 3 [bedrooms - bathrooms] - MAE trung bình:", sum(mae_scores)/len(mae_scores))
w = last_model.linear.weight.data.numpy()
b = last_model.linear.bias.data.numpy()

print("Trọng số w cuối cùng:", w)
print("Bias b cuối cùng:", b)